# Supervised Fine Tuning

- Once the base model (foundation model) is trained for text completion task, it is time for `Post-Training`
- It is the step in which the model will get capability of following instructions and conversing
- Post-training consists of two steps:
    - Supervised Fine Tuning (SFT)
    - Preference Fine Tuning (PFT)
- Supervised Fine-Tuning (SFT) - Training the model on curated, instruction-based data.
- Preference Fine-Tuning (RLHF, DPO, PPO) - Training the model to align better with human preference

In order to perform the SFT, there are three approaches:
- Full fine tuning
- LoRA
- QLoRA

In this notebook, QLoRA implemented to perform the SFT on phi-2 model using OpenAssistant dataset

---

## Install Dependencies

In [1]:
!pip install -q torch transformers accelerate bitsandbytes datasets peft trl pytorch-lightning tensorboard einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

## Model Training
Train the Phi-2 base model on [OpenAssistant](https://huggingface.co/datasets/OpenAssistant/oasst1?row=0) dataset using QLoRA

In [2]:
!python training.py --max_epochs 1 --no_validation

2025-03-06 14:28:26.596164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741271306.939142    1279 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741271307.035754    1279 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 14:28:27.762034: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Setting batch size to 1 to save memory
Set gradient accumulation to 16
Set dataloader workers to 0 (main process only

## Inference

In [11]:
!python inference.py --model_path ./checkpoints/last.ckpt --base_model microsoft/phi-2 --example_prompts

2025-03-06 14:55:45.923228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741272945.945246    8331 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741272945.951809    8331 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading tokenizer from microsoft/phi-2...
Loading model from ./checkpoints/last.ckpt...
Expected files not found. Available files in the directory:
Error loading model: [Errno 20] Not a directory: './checkpoints/last.ckpt'

Fallback: trying to load from base model path and merge with adapters...
Loading checkpoint shards: 100% 2/2 [00:29<00:00, 14.87s/it]
Using base model as fallback
Model loaded successfully and moved to cuda!
Using

---
---

## Train QLoRA Adapters

In [16]:
!python training.py --max_epochs 1 --save_adapters_only

2025-03-06 15:20:39.384177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741274439.421007   14680 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741274439.432251   14680 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

Found existing checkpoint: checkpoints/last.ckpt
Resume training from checkpoint? (y/n): n
Setting batch size to 1 to save memory
Set gradient accumulation to 16
Set dataloader workers to 0 (main process only)
DeepSpeed not available - using standard training
Enabling extreme memory saving options...
Loading checkpoint shards: 100% 2/2 [00:30<00:00, 15.14s/it]
trainable params: 10,485,760 || all params: 2,790,169,600 || trainable%: 

## Inference with QLoRA Adapters

In [21]:
!python inference.py --model_path ./adapters --base_model microsoft/phi-2 --use_qlora --example_prompts

2025-03-06 15:48:41.482040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741276121.511710   21842 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741276121.520966   21842 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading tokenizer from microsoft/phi-2...
Loading base model microsoft/phi-2...
Using 4-bit quantization (QLoRA)
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100% 2/2 [00:18<00:00,  9.16s/it]
Loading QLoRA adapters from ./adapters...
Converting normalization layers to float32 for stability
Model loaded successfully and moved to cuda!
Using 5 example prompts

Generating respons